In [8]:
import folium
import pandas as pd


In [9]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [10]:
import requests

def download(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
            
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

download(URL,'spacex_launch_geo.csv')

In [11]:
df = pd.read_csv('spacex_launch_geo.csv')


df_sites = df.groupby(['Launch Site'])['class'].mean()

#Compute success rate by each lauch site
df_sites

Launch Site
CCAFS LC-40     0.269231
CCAFS SLC-40    0.428571
KSC LC-39A      0.769231
VAFB SLC-4E     0.400000
Name: class, dtype: float64

In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
nasa_coordinate = [29.559684888503615, -95.0830971930759]

site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

nasa = folium.CircleMarker(nasa_coordinate, popup="NASA Johnson Space Center")

site_map.add_child(nasa)
site_map



In [13]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
df_sites = df[['Launch Site', 'Lat', 'Long', 'class']]

df_grouped = df_sites.groupby(['Launch Site'], as_index=False).first()

latitudes = list(df_grouped['Lat'])
longitudes = list(df_grouped['Long'])
labels = list(df_grouped['Launch Site'])

for x, y, l in zip(latitudes, longitudes, labels):
    folium.Circle([x,y], popup=l).add_to(site_map)
    
site_map
    


In [14]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
#circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
circle = folium.Circle(nasa_coordinate, popup = "NASA Johnson Space Center", radius=1000, color='#d35400', fill=True)

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.Marker(nasa_coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC'))
#site_map.add_child(circle)
#site_map.add_child(marker)

circle.add_to(site_map)
marker.add_to(site_map)

site_map

In [15]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

df_sites = df[['Launch Site', 'Lat', 'Long', 'class']]

df_grouped = df_sites.groupby(['Launch Site'], as_index=False).first()

#Mark the success/failed launches for each site on the map
latitudes = list(df_grouped['Lat'])
longitudes = list(df_grouped['Long'])
label1 = list(df_grouped['Launch Site'])
label2 = list(df_sites.groupby(['Launch Site'])['class'].mean())
for x, y, l1, l2 in zip(latitudes, longitudes, label1, label2):
    folium.Circle([x,y] ,  color='red', fill=True).add_to(site_map)
    folium.Marker([x,y], color='blue', fill=True ,   icon=DivIcon(icon_size=(40,40),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>Site: {}, Success Rate :{}</b></div>'.format(l1,l2))).add_to(site_map)
#, icon=DivIcon(icon_size=(40,40),icon_anchor=(-10,-10), html='<div style="font-size: 12; color:#d35400;"><p>%s</p></div>' % l1 )    
site_map

In [16]:
#Mark the success/failed launches for each site on the map

from folium import plugins

nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

sites = plugins.MarkerCluster().add_to(site_map)

latitudes = df_grouped['Lat']
longitudes = df_grouped['Long']
label1 = df_grouped['Launch Site']
label2 = df_sites.groupby(['Launch Site'])['class'].mean()

for x, y, l1, l2 in zip(latitudes, longitudes, label1, label2):
    folium.Marker(location=[x,y],popup='{} success rate: {}'.format(l1,l2)).add_to(sites)
    


site_map


In [17]:
#another more visually appealing way of depicting the successful outcomes

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

#create a marker cluster object
marker_cluster = plugins.MarkerCluster().add_to(site_map)

# Function to assign color 
def assign_marker_color(column):
    if column == 1:
        return 'green'
    else:
        return 'red'
    
df['marker_color'] = df['class'].apply(assign_marker_color)

for index, row in df.iterrows():
    # create and add a Marker cluster to the site map
    coordinate = [row['Lat'], row['Long']]
    folium.Marker(coordinate, icon=folium.Icon(color='white',icon_color=row['marker_color'])).add_to(marker_cluster)


site_map



In [18]:

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [21]:
from math import sin, cos, sqrt, atan2, radians

#This function allows you to calculate distance between two points on the earth
# using their coordinates.
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance



In [22]:
#let us visualize the distance from the near coastline
Launch_site_lat=28.56326
Launch_site_lng= -80.5768
coastal_point_lat = 28.5632
coastal_point_lng = -80.5679

nearest_distance = calculate_distance(Launch_site_lat, Launch_site_lng, coastal_point_lat, coastal_point_lng)

#, icon=DivIcon(icon_size=(40,40),icon_anchor=(-10,-10), html='<div style="font-size: 12; color:#d35400;"><p>%s</p></div>' % l1 )    
site_map = folium.Map([Launch_site_lat,Launch_site_lng], zoom_start=15)

my_launch_site = folium.Marker([Launch_site_lat,Launch_site_lng], 
                        
                       color='red', 
                       fill=True,
                      icon = DivIcon(icon_size=(40,40),
                                icon_anchor=(0,0),
                                  html='<divstyle="font-size: 12; color:#d35400;"><b>%s</b></div>' % "CCAFS SLC-40")
)

coastal_point = folium.Marker([coastal_point_lat,coastal_point_lng], 
                     
                       color='red', 
                       fill=True,
                      icon = DivIcon(icon_size=(40,40),
                                    icon_anchor=(0,0),
                                    html='<divstyle="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(nearest_distance))
)

#site_map.add_child(my_launch_site)
#site_map.add_child(coastal_point)

my_launch_site.add_to(site_map)
coastal_point.add_to(site_map)

coordinates = [[Launch_site_lat, Launch_site_lng], [coastal_point_lat, coastal_point_lng]]

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)